

## USE CASE 1-IMSP 1st DATA Science school

We recall the use case here

You are a software developper and you want to provide your clients with an application that predicts the fees and related waiting delays for they medical appointments. Clients can feed delay to predict the fees . 


Predictions will be based on data collected from different practitioners in a pilot city during a 6 months time. 


Let's start By Setting the working environment

# GO AHEAD and mount your Google Drive folder from this code

In [0]:
from google.colab import drive
drive.mount("/content/drive/")

Change your working directory to access your data. 

In [0]:
import os
os.chdir("drive/My Drive/ColabDev")

In [0]:
!ls
!pip install xlrd

In [0]:
!ls

**/ !!!!!! \ Note that the previous commands should be run only once for environment configuration  / !!!!!! \\.**

Use case implemetation can now start with defining the required packages in this use case.

# Data visualisation and preprocessing


We start with Data visualization and preprocessing before building out our model. You will find terms like pipeline or Data flow to define this process. It is crutial for the two following reasons:
-	You need to know what type of data you are dealing with and which feature are important for the task you are trying to achieve. 
-	You need to prepare your data to be efficiently usable during the training phase. Not just load and train. Normalization, One hot encoding, String Classification, Data augmentation and Class indexing are among the usual preparation items.


In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import seaborn as sns

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Let's load the data

In [0]:
#Enter you code here to load the data you can use pd.read_excel? plus enter have the description of the function

## *solution*

In [0]:
raw_df = pd.read_excel("DATASETS/child_doctor_appointments.xlsx", sheetname = "Pédiatres")
raw_df.head()

In [0]:
raw_df.info()#Gives a flate description of the data

Above we get the description of the data content which guide us in the way we will exploit them. Note at first place the "Ville" column which is the only non-numeric one. The next function will thus not be able to deal with it.

Let's get a statistical description on the data. What is your analysis?

## Data statistical summary
Let's get a statistical description on the data. What is your analysis?

In [0]:
raw_df.describe()

We focus next on the column of interest. Our objective is to precdict the fee overhead to pay compared to standard price of 28 EUR. What can we decide looking at the outputs?

Next we observe information about the relation between the colums. Seaborn package wraput nice plotting function that we will use here. 

Is this explained what you observe?

In [0]:
plt.figure(figsize=(5, 5))
sns.heatmap(raw_df.corr()[['Dépassement d\'honoraires']], annot=True, vmin=-1, vmax=1)

Show next the correlation matrix among your input features (Seaborn function is used here too)

In [0]:
plt.figure(figsize=(10, 8))
sns.heatmap(raw_df.corr(), annot=True, square=True, vmin=-1, vmax=1)

Next let's look at a histogramm of the data showing scales and regularity. You can also make a plain matrixplot of the data to visualize relations between the input features.

In [0]:
raw_df.hist(figsize=(10, 8))
plt.tight_layout()

In [0]:
#Use the Panda value_counts() function to list unique labels and their ratio counts in the data set
#Your code here

After getting all those insights about the data we are now ready to make the preparation phase.

## solution

In [0]:
(raw_df['Dépassement d\'honoraires'].value_counts())/raw_df['Dépassement d\'honoraires'].count()

## Data normalization and loading

Scikit-learn is often used in the data preparation phase. 

In particular what we will use is StandardScaler a simple normalization method. It individually scales the features such that they have zero mean and unit variance, so they all belong to a standard Normal(0, 1) distribution. Note that this doesn’t change the ordering of the feature values, it just changes the scale. It’s a simple yet extremely important trick.

In [0]:
df = raw_df.copy()
df = df.drop(['Tarif','Date appel', 'Date Rendez-vous'], axis=1)

std_fee = df['Dépassement d\'honoraires'].std(axis=0) 
mean_fee = df['Dépassement d\'honoraires'].mean(axis=0) 

ss = StandardScaler()
scale_features = ['Délai rendez-vous', 'Dépassement d\'honoraires']
df[scale_features] = ss.fit_transform(df[scale_features])

#Next we will encode the string column on the cities using one hot encoding feature from SkitLearn library.
df_cat = pd.get_dummies(df['Ville'])
df = df.drop('Ville', axis=1)
df_cat.head()

Let's look at our input data

In [0]:
df.head()

## Spliting the data
Scikit-learn has a convenient train_test_split function. We only need to specify the fraction of the test set, in our case 30%. But first we convert our data from pandas dataframe to numpy array using the values attribute of the dataframe.

In [0]:
X = df.drop('Dépassement d\'honoraires', axis=1).values
y = df['Dépassement d\'honoraires'].values

# Shuffle the training set
order = np.argsort(np.random.random(y.shape))
X = X[order]
y = y[order]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

## Build the NN model
Let's build our model. Here, we'll use a Sequential model with two densely connected hidden layers, and an output layer that returns a single, continuous value.

We use keras.layers library to create a Dense  layer.
Please change the second layer size to 64 units. 

In [0]:
def build_model():
  #How many layers and what sizes
  layer1_size = 6#12#4#64
  layer2_size = 6#4#64
  #How fast do you want to learn?
  learning_rate = 0.1
  
  model = keras.Sequential([
    #First layer
    keras.layers.Dense(layer1_size, activation=tf.nn.relu,
                       input_shape=(X_train.shape[1],)),
    #Second layer
    keras.layers.Dense(layer2_size, activation=tf.nn.relu),
    
    #Third layer
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(learning_rate)
  
  #Compile the model specifying loss function, choosen optimizer and evaluation metric
  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])#mape
  return model

model = build_model()
model.summary()

## Training the model

The model is trained for 500 epochs, and record the training and validation accuracy in the history object.

In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 500

#Fit the model parameters to the data while minimizing the loss. 
#Store training stats
history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])

In [0]:
history.history.keys()
mape = history.history['mean_absolute_error']
mape[-1]

Visualize the model's training progress using the stats stored in the history object. We want to use this data to determine how long to train before the model stops making progress.

In [0]:
def plot_history(history):
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error')
  plt.plot(history.epoch, np.array(history.history['mean_absolute_error']),
           label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
           label = 'Val loss')
  plt.legend()
  plt.ylim([0, 1])

plot_history(history)

We can now observe the model performance on the test set

In [0]:
[loss, mae] = model.evaluate(X_test, y_test, verbose=0)

print("Testing set Mean Abs Error: EUR{:7.2f}".format(mae * std_fee))

Let's now make some predictions

In [0]:
test_predictions = model.predict(X_test).flatten()

plt.scatter(y_test, test_predictions)
plt.xlabel('True Fees [EUR]')
plt.ylabel('Predictions [EUR]')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
_ = plt.plot([-100, 100], [-100, 100])

In [0]:
error = test_predictions - y_test
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")

What are your conclusions? 

We move now to another dataset with more data. In this dataset you have records of healthcare for a countrywise population. 

In this data set avalaible on Kaggle there are records of insurance charges for a bunch of people in the US from different profiles as well.

In [0]:
#Read the "insurance.csv" file from the DATASETS folder

#Your code here

In [0]:
#Transform the input data to input to your model. 
#Convert the categorical inputs

#Your code here

## solution

In [0]:
raw_df = pd.read_csv("DATASETS/insurance.csv")
raw_df.head()

In [0]:
#Convert string to int by "get_dummies"

sex_cat = pd.get_dummies(raw_df['sex'],drop_first=True,prefix='sex')
smoker_cat = pd.get_dummies(raw_df['smoker'],drop_first=True,prefix='smoker')
region_cat = pd.get_dummies(raw_df['region'],drop_first=True,prefix='region')

dummies = pd.concat([sex_cat,smoker_cat,region_cat],axis=1)

df = raw_df.copy()
#merge to the original table

df = pd.concat([df,dummies],axis=1)
df.head(10)

## Prepare data, input  and train

In [0]:
#Split dataset
#Drop columns containing string value
#Replace them with "get_dummies"
X = df.drop(columns=['charges','sex','smoker','region']).values
y = np.array(df['charges'])

# Shuffle the training set
order = np.argsort(np.random.random(y.shape))
X = X[order]
y = y[order]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [0]:
#Build your model and train it. You can look above as cheating sheet

## Solution

In [0]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.01)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()

# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 500

# Store training stats
history = model.fit(X_train, y_train, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])

## Plotting the outputs

In [0]:
def plot_history(history):
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error')
  plt.plot(history.epoch, np.array(history.history['mean_absolute_error']),
           label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
           label = 'Val loss')
  plt.legend()
  plt.ylim([0, 20000])

plot_history(history)

In [0]:
test_predictions = model.predict(X_test).flatten()

plt.scatter(y_test, test_predictions)
plt.xlabel('True Fees [EUR]')
plt.ylabel('Predictions [EUR]')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
_ = plt.plot([0, 60000], [0, 60000])

In [0]:
df.head()

## Prediction examples

Let's imagine 4 different people and see what charges on health care will be for them.

1. **Essombe**: 19 years old, BMI 27.9, has no children, smokes, from northwest region.

2. **Ishola**: 60 years old. BMI 40, 4 children, doesn't smoke, from soutwest region.

3. **Khady**: 40 years old, BMI 50, 2 children, doesn't smoke, from southeast region.

4. **Marcos**: 30 years old. BMI 31.2, no children, doesn't smoke, from northeast region.


In [0]:
test_pop = pd.DataFrame([{'age' : 19,'sex' : "Male",'bmi' : 27.9,'children' : 0,'smoker' : "yes",'region' : "northwest"}, 
                        {'age' : 60, 'sex' : "Male",'bmi' : 40, 'children' : 4,'smoker' : "no", 'region' : "soutwest"},
                        {'age' : 40, 'sex' : "Female",'bmi' : 50, 'children' : 2,'smoker' : "no", 'region' : "southeast"},                       
                        {'age' : 30, 'sex' : "Male",'bmi' : 31.2,'children' : 0,'smoker' : "no", 'region' : "northeast"}],index = ["Bob","John","Lisa","Terry"])

sex_cat = pd.get_dummies(test_pop['sex'],drop_first=True,prefix='sex')
smoker_cat = pd.get_dummies(test_pop['smoker'],drop_first=True,prefix='smoker')
region_cat = pd.get_dummies(test_pop['region'],drop_first=True,prefix='region')

dummies = pd.concat([sex_cat,smoker_cat,region_cat],axis=1)

test_pop = test_pop.drop(columns=['sex','smoker','region'])
#merge to the original table

test_pop = pd.concat([test_pop,dummies],axis=1)
test_pop.head()

In [0]:
prediction = model.predict(np.array(test_pop)).flatten()
print("Health care charges for Essombe : EUR {:7.2f}".format(round(prediction[0], 2)))
print("Health care charges for Ishola : EUR {:7.2f}".format(round(prediction[1], 2)))
print("Health care charges for Khady : EUR {:7.2f}".format(round(prediction[2], 2)))
print("Health care charges for Marcos : EUR {:7.2f}".format(round(prediction[3], 2)))